# Setting the Catalog

We are switching to the `workspace` catalog.  
In Unity Catalog, all tables are organized under catalogs and schemas.  
Here, we use the `workspace` catalog and default schema.


In [0]:
USE CATALOG workspace;

# Creating the Delta Lake Table: `employees`

We are creating a Delta table named `employees` with:

- `id` (Unique Employee ID)
- `name` (Employee Name)
- `salary` (Employee Salary)

Delta Lake format ensures:

- ACID transactions
- Versioning
- Scalable metadata management


In [0]:
-- Create a delta employees table
CREATE OR REPLACE TABLE employees
  (id INT, name STRING, salary DOUBLE)
USING DELTA;

### Exploring the Created Table in Catalog

After running the `CREATE TABLE` command:

1. Navigate to the **Catalog** tab (left sidebar).
2. Under **workspace → default**, you'll see the newly created **employees** table.
3. Click on **employees** to view the table's schema (columns: `id`, `name`, `salary`).

---

### Additional Tabs in Catalog:

- **Sample Data:**  
  Preview a sample of the table’s data (even before querying).

- **Details:**  
  Shows metadata like:
  - Storage location (DBFS path)
  - Table format (Delta by default)
  - Created by and timestamps

- **Permissions:**  
  Manage access permissions for the table (who can read/write).

- **History:**  
  Displays the full version history of the table (creation, inserts, updates).

- **Lineage:**  
  Visual representation of upstream and downstream data dependencies (where data comes from and where it goes).

- **Insights:**  
  Basic stats and information about the table contents (e.g., row count, size).

- **Quality (if enabled):**  
  Shows applied data quality checks (constraints, expectations, validations).

---

> **Tip:**  
Use the Catalog interface for easy table management without writing SQL, especially useful for inspecting structure, tracking history, and setting permissions!


# Inserting Data into the Delta Table

We insert Sanrio character employees in multiple transactions.  
Each transaction creates a separate Delta data file (since Delta Lake uses **copy-on-write**).

**Why multiple transactions?**

- Demonstrates Delta's ability to handle multiple files (for parallelism & scalability).

In [0]:
-- NOTE: With latest Databricks Runtimes, inserting few records in single transaction is optimized into single data file.
-- For this demo, we will insert the records in multiple transactions in order to create 4 data files.

INSERT INTO employees
VALUES 
  (1, "Hello Kitty", 3500.0),
  (2, "My Melody", 4020.5);

INSERT INTO employees
VALUES
  (3, "Cinnamoroll", 2999.3),
  (4, "Badtz-Maru", 4000.3);

INSERT INTO employees
VALUES
  (5, "Kuromi", 2500.0);

INSERT INTO employees
VALUES
  (6, "Keroppi", 6200.3)

-- NOTE: When executing multiple SQL statements in the same cell, only the last statement's result will be displayed in the cell output.

In [0]:
--View all the data in the employees table
SELECT * FROM employees

# Exploring Table Metadata

We use `DESCRIBE DETAIL` to inspect the Delta table's metadata.

**Key info:**

- Table format: Delta
- Creation time
- Schema details
- Number of data files
- Managed by Unity Catalog (Location may show blank, storage is abstracted)

In [0]:
--Use the DESCRIBE DETAIL command on the table to view some metadata information about the table
DESCRIBE DETAIL employees

# Updating Records in Delta Table

We update salaries for employees whose names start with "K".  

**Delta Lake behavior:**

- Writes new Parquet files (copy-on-write)
- Old files are marked invalid
- Transaction log records the update ensuring **ACID compliance**

In [0]:
UPDATE employees 
SET salary = salary + 100
WHERE name LIKE "K%";

# Viewing Updated Records

Query the `employees` table again to confirm the salary updates.

In [0]:
SELECT * FROM employees;

# Checking Table Metadata Post-Update

Run `DESCRIBE DETAIL` again to view:

- Current version metadata
- Confirmation of Delta format
- Info remains consistent after the update


In [0]:
DESCRIBE DETAIL employees;

# Exploring Table History

Delta Lake automatically tracks:

- Table creation
- Insert operations
- Update operations

`DESCRIBE HISTORY` shows a versioned audit trail with timestamps & user info.


In [0]:
DESCRIBE HISTORY employees;